In [2]:
import os

In [3]:
%pwd

'/workspaces/Chicken-Disease-Classification/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/workspaces/Chicken-Disease-Classification'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):

            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self)->PrepareCallbacksConfig:
          config=self.config.prepare_callbacks
          model_ckpt_dir=os.path.dirname(config.checkpoint_model_filepath)
          create_directories([
                Path(model_ckpt_dir),
                Path(config.tensorboard_root_log_dir)
          ])

          prepare_callback_config=PrepareCallbacksConfig(
                root_dir=Path(config.root_dir),
                tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
                checkpoint_model_filepath=str(config.checkpoint_model_filepath)
          )

          return prepare_callback_config

In [9]:
import os,pytz
import urllib.request as request
import zipfile
import tensorflow as tf
import time
from datetime import datetime

2024-08-09 05:57:53.308728: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 05:57:53.376070: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 05:57:53.377829: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-09 05:57:54.458909: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
class PrepareCallback:
    def __init__(self,config:PrepareCallbacksConfig):
        self.config=config
    
    @property
    def _create_tf_callbacks(self):
        timezone_ind= pytz.timezone('Asia/Kolkata')
        ist_local = datetime.now(timezone_ind)
        timestamp=ist_local.strftime('%d_%m_%Y_%H_%M_%S')
        tb_running_log_dir=os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callback(self):
        return [
            self._create_tf_callbacks,
            self._create_ckpt_callbacks
        ]


In [11]:
try:
    config=ConfigurationManager()
    prepare_callbacks_config=config.get_prepare_callbacks_config()
    prepare_callbacks=PrepareCallback(config=prepare_callbacks_config)
    callback_list=prepare_callbacks.get_tb_ckpt_callback()
except Exception as e:
    raise e

[2024-08-09 05:57:55,904] common.py:30 common:read_yaml() cnnClassifierLogger - INFO - yaml file: /workspaces/Chicken-Disease-Classification/config/config.yaml loaded successfully
[2024-08-09 05:57:55,906] common.py:30 common:read_yaml() cnnClassifierLogger - INFO - yaml file: /workspaces/Chicken-Disease-Classification/params.yaml loaded successfully
[2024-08-09 05:57:55,907] common.py:48 common:create_directories() cnnClassifierLogger - INFO - created dictionary at: artifacts
[2024-08-09 05:57:55,908] common.py:48 common:create_directories() cnnClassifierLogger - INFO - created dictionary at: artifacts/prepare_callbacks/checkpoint_dir
[2024-08-09 05:57:55,909] common.py:48 common:create_directories() cnnClassifierLogger - INFO - created dictionary at: artifacts/prepare_callbacks/tensorboard_log_dir
